# Projeto: Engenharia de Dados e Garantia de Qualidade no Conjunto de Dados do Airbnb no Rio de Janeiro

#### Grupo5:
Mariana,
Gabriella,
Manu,
Fabio

In [ ]:
# instalar adaptador
# pip install psycopg2
# pip install pandera


In [1]:
# importar bibliotecas
import pandas as pd
import psycopg2
import numpy as np
import pandera as pa
from datetime import datetime, timedelta
from pandera import Check
from pandera.typing import DataFrame, Series

In [2]:
# mostrar todas as linhas e colunas dos dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Camada Bronze
A camada bronze salva no banco de dados do PostgreSQL contém os dados em seu formato original

In [44]:
#estabelecer conexão com a camada bronze
try:
    conn = psycopg2.connect(
        host="localhost",
        database="bronze_layer",
        user="postgres",
        password="7596"
    )
    print("Conexão bem-sucedida!")
except Exception as e:
    print("Erro ao conectar:", e)

Conexão bem-sucedida!


In [37]:
# criar cursor
cur_review = conn.cursor()
# Executar consulta para exibir todas as colunas de uma tabela
cur_review.execute("""
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_name = 'reviews_table_bronze'
""")

# Obter e exibir o resultado
columns = cur_review.fetchall()
for column in columns:
    print("Coluna:", column[0], "| Tipo de dado:", column[1])

Coluna: listing_id | Tipo de dado: text
Coluna: id | Tipo de dado: text
Coluna: date | Tipo de dado: text
Coluna: reviewer_id | Tipo de dado: text
Coluna: reviewer_name | Tipo de dado: text
Coluna: comments | Tipo de dado: text


In [5]:
#selecionar conteúdo da tabela
cur_review.execute('SELECT * FROM reviews_table_bronze')
#selecionar todas as linhas
rows_review = cur_review.fetchall()
#transformar em dataframe
reviews = pd.DataFrame(rows_review, columns=['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'])

In [6]:
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...
1,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...
2,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...
3,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r<b...
4,17878,147594,2010-12-01,219338,David,The apt was nice and in a great location only ...


In [45]:
# criar cursor
cur_listings = conn.cursor()
# Executar consulta para exibir todas as colunas de uma tabela
cur_listings.execute("""
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_name = 'listings_table_bronze'
""")

# Obter e exibir o resultado
columns = cur_listings.fetchall()
for column in columns:
    print("Coluna:", column[0], "| Tipo de dado:", column[1])

Coluna: id | Tipo de dado: text
Coluna: listing_url | Tipo de dado: text
Coluna: scrape_id | Tipo de dado: text
Coluna: last_scraped | Tipo de dado: text
Coluna: source | Tipo de dado: text
Coluna: name | Tipo de dado: text
Coluna: description | Tipo de dado: text
Coluna: neighborhood_overview | Tipo de dado: text
Coluna: picture_url | Tipo de dado: text
Coluna: host_id | Tipo de dado: text
Coluna: host_url | Tipo de dado: text
Coluna: host_name | Tipo de dado: text
Coluna: host_since | Tipo de dado: text
Coluna: host_location | Tipo de dado: text
Coluna: host_about | Tipo de dado: text
Coluna: host_response_time | Tipo de dado: text
Coluna: host_response_rate | Tipo de dado: text
Coluna: host_acceptance_rate | Tipo de dado: text
Coluna: host_is_superhost | Tipo de dado: text
Coluna: host_thumbnail_url | Tipo de dado: text
Coluna: host_picture_url | Tipo de dado: text
Coluna: host_neighbourhood | Tipo de dado: text
Coluna: host_listings_count | Tipo de dado: text
Coluna: host_total_lis

In [46]:
#selecionar conteúdo da tabela
cur_listings.execute('SELECT * FROM listings_table_bronze')
#selecionar todas as linhas
rows_listings = cur_listings.fetchall()
#transformar em dataframe
listings = pd.DataFrame(rows_listings, columns=['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name', 'description', 'neighborhood_overview', 'picture_url', 'host_id',
'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate',
'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count',
'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed',
'neighbourhood_group_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bathrooms_text',
'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'calendar_last_scraped',
'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review', 'last_review', 'review_scores_rating',
'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
'review_scores_value', 'license', 'instant_bookable', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes',
'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'reviews_per_month'
])

In [47]:
listings.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,17878,https://www.airbnb.com/rooms/17878,20240627045056,2024-06-28,city scrape,"Very Nice 2Br in Copacabana w. balcony, fast WiFi",Please note that elevated rates applies for Ne...,This is the one of the bests spots in Rio. Bec...,https://a0.muscache.com/pictures/65320518/3069...,68997,https://www.airbnb.com/users/show/68997,Matthias,2010-01-08,"Rio de Janeiro, Brazil",I am a journalist/writer. Lived in NYC for ...,within an hour,100%,100%,t,https://a0.muscache.com/im/pictures/user/67b13...,https://a0.muscache.com/im/pictures/user/67b13...,Copacabana,2,5,"['email', 'phone']",t,t,"Rio de Janeiro, Brazil",Copacabana,None,-22.96599,-43.1794,Entire condo,Entire home/apt,5,1.0,1 bath,2,2,"[""Smoking allowed"", ""Essentials"", ""Air conditi...",$310.00,5,28,5,5,28,28,5.0,28.0,None,t,6,32,51,274,2024-06-28,319,23,1,2010-07-15,2024-06-08,4.71,4.77,4.64,4.84,4.91,4.77,4.67,None,f,1,1,0,0,1.88
1,25026,https://www.airbnb.com/rooms/25026,20240627045056,2024-06-28,city scrape,Beautiful Modern Decorated Studio in Copa,"**Fully renovated in Dec 2022, new kitchen, n...",Copacabana is a lively neighborhood and the ap...,https://a0.muscache.com/pictures/7c08fa4f-1d7b...,102840,https://www.airbnb.com/users/show/102840,Viviane,2010-04-03,"Rio de Janeiro, Brazil","Hi guys,\n\nViviane is a commercial photograph...",within an hour,100%,82%,t,https://a0.muscache.com/im/pictures/user/315dd...,https://a0.muscache.com/im/pictures/user/315dd...,Copacabana,1,5,"['email', 'phone']",t,t,"Rio de Janeiro, Brazil",Copacabana,None,-22.97735,-43.19105,Entire rental unit,Entire home/apt,3,1.0,1 bath,1,2,"[""Fast wifi \u2013 501 Mbps"", ""Essentials"", ""H...",$203.00,2,60,2,2,60,60,2.0,60.0,None,t,24,54,71,241,2024-06-28,291,23,1,2010-06-07,2024-06-02,4.73,4.71,4.79,4.82,4.92,4.84,4.62,None,f,1,1,0,0,1.70
2,220377,https://www.airbnb.com/rooms/220377,20240627045056,2024-06-27,city scrape,Suíte Casal (banheiro privativo),The apartment is cozy and well located. The r...,Tijuca is a residential neigbohood in Rio clos...,https://a0.muscache.com/pictures/992d8e44-0ef1...,1142424,https://www.airbnb.com/users/show/1142424,Taciana,2011-09-11,None,,within an hour,100%,47%,f,https://a0.muscache.com/im/pictures/user/c24dc...,https://a0.muscache.com/im/pictures/user/c24dc...,Tijuca,3,3,['phone'],t,t,"Rio de Janeiro, Brazil",Tijuca,None,-22.9288,-43.24046,Private room in rental unit,Private room,2,1.0,1 private bath,1,1,"[""Essentials"", ""Air conditioning"", ""Hangers"", ...",$220.00,1,365,1,1,1125,1125,1.0,1125.0,None,t,9,29,41,283,2024-06-27,5,1,0,2011-11-08,2024-03-28,5.0,5.0,5.0,5.0,4.8,4.8,5.0,None,f,3,0,3,0,0.03
3,35764,https://www.airbnb.com/rooms/35764,20240627045056

In [40]:
# criar cursor
cur_calendar = conn.cursor()
# Executar consulta para exibir todas as colunas de uma tabela
cur_calendar.execute("""
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_name = 'calendar_table_bronze'
""")

# Obter e exibir o resultado
columns = cur_calendar.fetchall()
for column in columns:
    print("Coluna:", column[0], "| Tipo de dado:", column[1])

Coluna: listing_id | Tipo de dado: text
Coluna: date | Tipo de dado: text
Coluna: available | Tipo de dado: text
Coluna: price | Tipo de dado: text
Coluna: adjusted_price | Tipo de dado: text
Coluna: minimum_nights | Tipo de dado: text
Coluna: maximum_nights | Tipo de dado: text


In [ ]:
# #contar as entradas da tabela
# cur_calendar.execute('SELECT COUNT(*) FROM calendar_table_bronze')
# #selecionar todas as linhas
# nu = cur_calendar.fetchall()
# # Imprime o resultado
# print(f"Número de registros na tabela: {nu[0][0]}")

In [11]:
#selecionar conteúdo da tabela com limite para não quebrar o kernel
cur_calendar.execute('SELECT * FROM calendar_table_bronze LIMIT 100000')
# #selecionar conteúdo da tabela
# cur_calendar.execute('SELECT * FROM calendar_table_bronze')
#selecionar todas as linhas
rows_calendar = cur_calendar.fetchall()
#transformar em dataframe
calendar = pd.DataFrame(rows_calendar, columns=['listing_id', 'date', 'available', 'price', 'adjusted_price', 'minimum_nights', 'maximum_nights'])

In [12]:
calendar.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,1479860,2025-01-20,f,$150.00,None,1,90
1,1479860,2025-01-21,f,$150.00,None,1,90
2,1479860,2025-01-22,f,$150.00,None,1,90
3,1479860,2025-01-23,f,$150.00,None,1,90
4,1479860,2025-01-24,f,$150.00,None,1,90


### Camada Silver
A camada Silver contém dados padronizados, limpos e métricas de qualidade implementadas

Data Clean - Camada Silver: <br>
- Identifique e lide com valores ausentes, duplicatas e outliers nos dados brutos da camada "bronze". <br>
- Padronize e limpe os nomes das colunas, convertendo-os em um formato consistente. <br>
- Realize uma limpeza textual em campos, como descrições de propriedades, removendo caracteres especiais e erros de digitação.

Data Quality - Camada Silver:
- Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".
- Implemente verificações para garantir que os dados da camada "silver" estejam em conformidade com essas métricas.
- Estabeleça um sistema de monitoramento contínuo da qualidade dos dados da camada "silver".

In [13]:
#converter reviews para os tipos corretos
reviews[['listing_id', 'id', 'reviewer_id']] = reviews[['listing_id', 'id', 'reviewer_id']].apply(pd.to_numeric)
reviews['date'] = pd.to_datetime(reviews['date'])


In [14]:

#Nulos e duplicados
reviews = reviews.dropna(how="all", axis=1).dropna(how="all")
reviews = reviews.drop_duplicates()


In [15]:

#remover outros caracteres especiais
remover = [']', '[', '"', '\n', '*', '\r', '<br/>']

for caractere in remover:
    for coluna in reviews.columns:
        if reviews[coluna].dtype == 'object':
            reviews[coluna] = reviews[coluna].str.replace(caractere, '', regex=False)

In [16]:
# Verificar os tipos de dados de cada coluna
print(reviews.dtypes)
# Verificar se ainda há valores nulos
print("Valores nulos por coluna:")
print(reviews.isnull().sum())


listing_id                int64
id                        int64
date             datetime64[ns]
reviewer_id               int64
reviewer_name            object
comments                 object
dtype: object
Valores nulos por coluna:
listing_id       0
id               0
date             0
reviewer_id      0
reviewer_name    0
comments         0
dtype: int64


In [17]:
# Verificar se há duplicados restantes
print("Total de duplicados:", reviews.duplicated().sum())


Total de duplicados: 0


In [18]:

# Exibir uma amostra de registros nas colunas 'object' para verificar a limpeza dos caracteres especiais
print(reviews.select_dtypes(include='object').head(5))

  reviewer_name                                           comments
0           Tia  This apartment is in a perfect location -- two...
1          Mimi  we had a really great experience staying in Ma...
2           Jan  Staying in Max appartment is like living in a ...
3         Orene  In general very good and reasonable price.It i...
4         David  The apt was nice and in a great location only ...


In [19]:
#Nulos e duplicados
calendar = calendar.dropna(how="all", axis=1).dropna(how="all")
calendar = calendar.drop_duplicates()


In [20]:

#converter calendar para os tipos corretos e remover caracteres especiais
calendar['price'] = calendar['price'].str.replace('$','', regex=False) #adjusted_price/price
calendar['price'] = calendar['price'].str.replace(',','', regex=False) #adjusted_price/price


In [21]:

# Converter para número
calendar[['listing_id', 'minimum_nights', 'maximum_nights','price']] = calendar[['listing_id', 'minimum_nights', 'maximum_nights','price']].apply(pd.to_numeric) #adjusted_price/price

# Converter 'date' para data
calendar['date'] = pd.to_datetime(calendar['date'])

# Converter "t" 'f' para True e False
calendar['available'] = np.where(calendar['available']=="t",True,False)


In [22]:

## Tratar valores nulos
for col in calendar.select_dtypes(include = "number").columns:
    calendar[col] = calendar[col].fillna(calendar[col].mean()).astype('int')


In [23]:
# Verificar os tipos de dados de cada coluna
print(calendar.dtypes)
# Verificar se ainda há valores nulos
print("Valores nulos por coluna:")
print(calendar.isnull().sum())


listing_id                 int64
date              datetime64[ns]
available                   bool
price                      int64
minimum_nights             int64
maximum_nights             int64
dtype: object
Valores nulos por coluna:
listing_id        0
date              0
available         0
price             0
minimum_nights    0
maximum_nights    0
dtype: int64


In [24]:

# Verificar se há duplicados restantes
print("Total de duplicados:", calendar.duplicated().sum())


Total de duplicados: 0


In [48]:

listings = listings[listings.host_location=='Rio de Janeiro, Brazil'] # Filtrar Rio de Janeiro

#Nulos e duplicados
listings = listings.dropna(how="all", axis=1).dropna(how="all")
listings = listings.drop_duplicates()


In [49]:
#converter listings para os tipos corretos e remover caracteres especiais
converter_numerico = [
    'id', 'host_id', 'host_listings_count', 'host_total_listings_count',
    'accommodates', 'bathrooms', 'bedrooms', 'beds', 'minimum_nights',
    'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
    'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm',
    'maximum_nights_avg_ntm', 'availability_30', 'availability_60',
    'availability_90', 'availability_365', 'number_of_reviews',
    'number_of_reviews_ltm', 'number_of_reviews_l30d', 'review_scores_rating',
    'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin',
    'review_scores_communication', 'review_scores_location', 'review_scores_value',
    'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes',
    'calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms', 'reviews_per_month'
]

converter_data = [
    'last_scraped', 'host_since', 'first_review', 'last_review'
]

#converter os tipos conforme as listas acima
listings[converter_numerico] = listings[converter_numerico].apply(pd.to_numeric)
# listings[converter_data] = listings[converter_data].apply(pd.to_datetime)


In [50]:
# Converter a coluna first_review para o tipo datetime, e substituir erros por NaT (Not a Time)
listings['first_review'] = pd.to_datetime(listings['first_review'], errors='coerce')

# Substituir NaT por None para compatibilidade com SQL (None é o equivalente a NULL)
listings['first_review'] = listings['first_review'].where(listings['first_review'].notna(), None)

In [51]:

#corrigir a coluna de price que tinha caractere especial
listings['price'] = listings['price'].str.replace('$','', regex=False)
listings['price'] = listings['price'].str.replace(',','', regex=False).astype(float)

#existem valores N/A nas colunas host_response_rate e host_acceptance_rate; é prevciso alterar para NaN
listings['host_response_rate'] = listings['host_response_rate'].replace('N/A', np.NaN)
listings['host_acceptance_rate'] =listings['host_acceptance_rate'].replace('N/A', np.NaN)

#converter colunas com % para formato adequado
listings['host_response_rate'] = listings['host_response_rate'].str.replace('%', '', regex=False).astype(float) / 100
listings['host_acceptance_rate'] = listings['host_acceptance_rate'].str.replace('%', '', regex=False).astype(float) / 100


In [52]:

#algumas colunas de listings estão com vazio 'None', vou mudar para NaN
contem_vazios = ['description', 'neighborhood_overview', 'host_location', 'host_response_rate',
'host_acceptance_rate', 'host_is_superhost', 'host_neighbourhood', 'neighbourhood',  'bathrooms', 'bathrooms_text', 'bedrooms',
'beds', 'price', 'has_availability', 'first_review',
'last_review', 'review_scores_rating', 'review_scores_accuracy',
'review_scores_cleanliness', 'review_scores_checkin',
'review_scores_communication', 'review_scores_location',
'review_scores_value', 'reviews_per_month']

listings[contem_vazios] = listings[contem_vazios].fillna(np.nan)

#remover outros caracteres especiais
remover = [']', '[', '"', '\n', '*']

for caractere in remover:
    for coluna in listings.columns:
        if listings[coluna].dtype == 'object':
            listings[coluna] = listings[coluna].str.replace(caractere, '', regex=False)


In [53]:
# Converter True/False
for col in ['host_is_superhost','instant_bookable']:
    listings[col] = np.where(listings[col] =="t",True,False)

# Ajustando a coluna de verificações
expand = listings['host_verifications'].str.split(',', expand=True, n=3)
for col in expand.columns:
    expand[col] = expand[col].str.replace("'",'').str.replace("]",'').str.replace("[",'')

novos_nomes = {expand.columns[0]: 'host_verification_1',
               expand.columns[1]: 'host_verification_2',
               expand.columns[2]: 'host_verification_3'}


expand.rename(columns=novos_nomes, inplace=True)

listings = pd.concat([listings,expand],axis=1).drop("host_verifications",axis=1)


Validação de Expectativas com Pandera - Camada Silver:
- Implemente validações adicionais usando Pandera nas camadas de dados da camada "silver".
- Monitore a qualidade dos dados da camada "silver" após cada transformação e ajuste os testes de acordo.

In [ ]:
# pip install pandera

In [ ]:
#validação Calendar
class CalendarSchema(pa.DataFrameModel):
    listing_id: pa.typing.Series[np.int64]
    date: pa.typing.Series[pd.Timestamp]
    available: pa.typing.Series[bool]
    price: pa.typing.Series[np.int64]
    minimum_nights: pa.typing.Series[np.int64]
    maximum_nights: pa.typing.Series[np.int64]


    # Verificar se a coluna minimum_nights não tem nenhum valor maior que a coluna maximum_nights
    @pa.check("minimum_nights")
    def check_minimum_nights_less_than_maximum(cls, series: pa.typing.Series) -> pa.typing.Series:
        return calendar["minimum_nights"] <= calendar["maximum_nights"]


In [ ]:

# Validação com Pandera
try:
    CalendarSchema.validate(calendar)
    print("Validação bem-sucedida!")
except pa.errors.SchemaError as e:
    print(f"Erro de validação: {e}")

In [ ]:
#validação reviews
class ReviewsSchema(pa.DataFrameModel):
    reviewer_name: pa.typing.Series[str]
    id: pa.typing.Series[int]
    listing_id: pa.typing.Series[int]
    date: pa.typing.Series[pd.Timestamp]

    # Validação para garantir que 'listing_id' seja único
    @pa.check("id")
    def check_id_unique(cls, series: pa.typing.Series) -> pa.typing.Series:
        return series.is_unique


In [ ]:

try:
    # Valide o DataFrame com o schema do Pandera
    ReviewsSchema.validate(reviews)
    print("Validação bem-sucedida!")
except pa.errors.SchemaError as e:
    print(f"Erro de validação: {e}")

In [ ]:
# Validação Listings
class ListingsSchema(pa.DataFrameModel):
    host_location: pa.typing.Series[str]
    id: pa.typing.Series[int]
    last_scraped: pa.typing.Series[pd.Timestamp]

    # Validação para 'host_location' estar na lista de locais esperados
    @pa.check("host_location")
    def check_host_location_in_set(cls, series: pa.typing.Series) -> pa.typing.Series:
        # Conjunto de locais permitidos
        allowed_locations = ["Rio de Janeiro, Brazil"]
        return series.isin(allowed_locations)

    # Validação para garantir que 'id' seja único
    @pa.check("id")
    def check_id_unique(cls, series: pa.typing.Series) -> pa.typing.Series:
        # Verificando se os valores são únicos
        return series.is_unique

    # Validação para 'last_scraped' estar dentro do intervalo de 12 meses
    @pa.check("last_scraped")
    def check_last_scraped_recent(cls, series: pa.typing.Series) -> pa.typing.Series:
        # Definindo o intervalo de 12 meses
        twelve_months_ago = pd.Timestamp(datetime.now() - timedelta(days=365))
        current_date = pd.Timestamp(datetime.now())

        # Verificando se os valores estão dentro do intervalo
        return (series >= twelve_months_ago) & (series <= current_date)


In [ ]:
try:
    # Valide o DataFrame com o schema do Pandera
    ListingsSchema.validate(listings.dropna(subset='host_location'))
    print("Validação bem-sucedida!")
except pa.errors.SchemaError as e:
    print(f"Erro de validação: {e}")

### Subir as tabelas Silver no dataframe do PostgreSQL

Tabela calendar_silver

In [ ]:
#criar uma nova tabela no postgre para calendar
create_table_query = """
CREATE TABLE IF NOT EXISTS calendar_silver (
    listing_id INT,
    date DATE,
    available BOOLEAN,
    price FLOAT,
    minimum_nights INT,
    maximum_nights INT
);
"""

cur_calendar.execute(create_table_query)
conn.commit()

In [ ]:
# Converter o DataFrame para uma lista de tuplas
data_to_insert = calendar.values.tolist()

# Inserir os dados na tabela 'calendar_silver'
insert_query = """
INSERT INTO calendar_silver (listing_id, date, available, price, minimum_nights, maximum_nights)
VALUES (%s, %s, %s, %s, %s, %s);
"""

cur_calendar.executemany(insert_query, data_to_insert)
conn.commit()

Tabela review_silver

In [ ]:
#criar tabela reviews_silver
create_reviews_table_query = """
CREATE TABLE IF NOT EXISTS reviews_silver (
    listing_id NUMERIC,
    id NUMERIC PRIMARY KEY,
    date DATE,
    reviewer_id NUMERIC,
    reviewer_name TEXT,
    comments TEXT
);
"""

cur_review.execute(create_reviews_table_query)
conn.commit()

In [ ]:
data_to_insert = reviews[['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments']].values.tolist()

insert_reviews_query = """
INSERT INTO reviews_silver (listing_id, id, date, reviewer_id, reviewer_name, comments)
VALUES (%s, %s, %s, %s, %s, %s)
ON CONFLICT (id) DO NOTHING;  -- Ignora se o id já existe
"""

cur_review.executemany(insert_reviews_query, data_to_insert)
conn.commit()

Tabela listings_silver

In [ ]:
create_table_query = """
CREATE TABLE IF NOT EXISTS listings_silver (
    id NUMERIC,
    listing_url TEXT,
    scrape_id TEXT,
    last_scraped TIMESTAMP,
    source TEXT,
    name TEXT,
    description TEXT,
    neighborhood_overview TEXT,
    picture_url TEXT,
    host_id NUMERIC,
    host_url TEXT,
    host_name TEXT,
    host_since TIMESTAMP,
    host_location TEXT,
    host_about TEXT,
    host_response_time TEXT,
    host_response_rate FLOAT,
    host_acceptance_rate FLOAT,
    host_is_superhost BOOLEAN,
    host_thumbnail_url TEXT,
    host_picture_url TEXT,
    host_neighbourhood TEXT,
    host_listings_count INT,
    host_total_listings_count INT,
    host_has_profile_pic TEXT,
    host_identity_verified TEXT,
    neighbourhood TEXT,
    neighbourhood_cleansed TEXT,
    latitude TEXT,
    longitude TEXT,
    property_type TEXT,
    room_type TEXT,
    accommodates INT,
    bathrooms FLOAT,
    bathrooms_text TEXT,
    bedrooms FLOAT,
    beds FLOAT,
    amenities TEXT,
    price FLOAT,
    minimum_nights INT,
    maximum_nights INT,
    minimum_minimum_nights INT,
    maximum_minimum_nights INT,
    minimum_maximum_nights INT,
    maximum_maximum_nights INT,
    minimum_nights_avg_ntm FLOAT,
    maximum_nights_avg_ntm FLOAT,
    has_availability TEXT,
    availability_30 INT,
    availability_60 INT,
    availability_90 INT,
    availability_365 INT,
    calendar_last_scraped TEXT,
    number_of_reviews INT,
    number_of_reviews_ltm INT,
    number_of_reviews_l30d INT,
    first_review TIMESTAMP,
    last_review TIMESTAMP,
    review_scores_rating FLOAT,
    review_scores_accuracy FLOAT,
    review_scores_cleanliness FLOAT,
    review_scores_checkin FLOAT,
    review_scores_communication FLOAT,
    review_scores_location FLOAT,
    review_scores_value FLOAT,
    instant_bookable BOOLEAN,
    calculated_host_listings_count INT,
    calculated_host_listings_count_entire_homes INT,
    calculated_host_listings_count_private_rooms INT,
    calculated_host_listings_count_shared_rooms INT,
    reviews_per_month FLOAT,
    host_verification_1 TEXT,
    host_verification_2 TEXT,
    host_verification_3 TEXT
);
"""

cur_listings.execute(create_table_query)
conn.commit()

In [31]:
cur_listings.execute("""
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_name = 'listings_silver'
""")

# Obter e exibir o resultado
columns = cur_listings.fetchall()
for column in columns:
    print("Coluna:", column[0], "| Tipo de dado:", column[1])

Coluna: host_is_superhost | Tipo de dado: boolean
Coluna: minimum_nights | Tipo de dado: integer
Coluna: maximum_nights | Tipo de dado: integer
Coluna: minimum_minimum_nights | Tipo de dado: integer
Coluna: maximum_minimum_nights | Tipo de dado: integer
Coluna: minimum_maximum_nights | Tipo de dado: integer
Coluna: maximum_maximum_nights | Tipo de dado: integer
Coluna: minimum_nights_avg_ntm | Tipo de dado: double precision
Coluna: maximum_nights_avg_ntm | Tipo de dado: double precision
Coluna: availability_30 | Tipo de dado: integer
Coluna: availability_60 | Tipo de dado: integer
Coluna: availability_90 | Tipo de dado: integer
Coluna: availability_365 | Tipo de dado: integer
Coluna: number_of_reviews | Tipo de dado: integer
Coluna: number_of_reviews_ltm | Tipo de dado: integer
Coluna: number_of_reviews_l30d | Tipo de dado: integer
Coluna: first_review | Tipo de dado: timestamp without time zone
Coluna: last_review | Tipo de dado: timestamp without time zone
Coluna: review_scores_ratin

In [55]:
data_to_insert = [
    [None if pd.isna(value) else value for value in row]
    for row in listings.values.tolist()
]

insert_query = """
INSERT INTO listings_silver (id, listing_url, scrape_id, last_scraped, source, name, description,
    neighborhood_overview, picture_url, host_id, host_url, host_name, host_since, host_location,
    host_about, host_response_time, host_response_rate, host_acceptance_rate, host_is_superhost,
    host_thumbnail_url, host_picture_url, host_neighbourhood, host_listings_count, host_total_listings_count,
    host_has_profile_pic, host_identity_verified, neighbourhood, neighbourhood_cleansed,
    latitude, longitude, property_type, room_type, accommodates, bathrooms, bathrooms_text, bedrooms, beds, amenities,
    price, minimum_nights, maximum_nights, minimum_minimum_nights, maximum_minimum_nights, minimum_maximum_nights,
    maximum_maximum_nights, minimum_nights_avg_ntm, maximum_nights_avg_ntm, has_availability,
    availability_30, availability_60, availability_90, availability_365, calendar_last_scraped, number_of_reviews,
    number_of_reviews_ltm, number_of_reviews_l30d, first_review, last_review, review_scores_rating, review_scores_accuracy,
    review_scores_cleanliness, review_scores_checkin, review_scores_communication, review_scores_location,
    review_scores_value, instant_bookable, calculated_host_listings_count, calculated_host_listings_count_entire_homes,
    calculated_host_listings_count_private_rooms, calculated_host_listings_count_shared_rooms, reviews_per_month,
    host_verification_1, host_verification_2, host_verification_3)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""


cur_listings.executemany(insert_query, data_to_insert)
conn.commit()


In [ ]:
# # Quantidade de colunas em listings
# num_colunas_listings = len(listings.columns)

# # Quantidade de elementos em cada sublista de data_to_insert
# num_colunas_data_to_insert = len(data_to_insert[0])  # Use o primeiro elemento como referência

# # Comparação
# if num_colunas_listings == num_colunas_data_to_insert:
#     print("O número de colunas corresponde!")
# else:
#     print(f"Quantidade de colunas não corresponde: listings tem {num_colunas_listings} colunas e data_to_insert tem {num_colunas_data_to_insert} colunas.")


In [ ]:
# # Lista de nomes de colunas do DataFrame
# colunas_listings = list(listings.columns)

# # Lista de nomes de colunas esperadas (use a ordem no SQL)
# colunas_sql = [
#     "id", "listing_url", "scrape_id", "last_scraped", "source", "name", "description",
#     "neighborhood_overview", "picture_url", "host_id", "host_url", "host_name", "host_since",
#     "host_location", "host_about", "host_response_time", "host_response_rate", "host_acceptance_rate",
#     "host_is_superhost", "host_thumbnail_url", "host_picture_url", "host_neighbourhood",
#     "host_listings_count", "host_total_listings_count", "host_has_profile_pic", "host_identity_verified",
#     "neighbourhood", "neighbourhood_cleansed", "latitude", "longitude", "property_type", "room_type",
#     "accommodates", "bathrooms", "bathrooms_text", "bedrooms", "beds", "amenities", "price",
#     "minimum_nights", "maximum_nights", "minimum_minimum_nights", "maximum_minimum_nights",
#     "minimum_maximum_nights", "maximum_maximum_nights", "minimum_nights_avg_ntm", "maximum_nights_avg_ntm",
#     "has_availability", "availability_30", "availability_60", "availability_90", "availability_365",
#     "calendar_last_scraped", "number_of_reviews", "number_of_reviews_ltm", "number_of_reviews_l30d",
#     "first_review", "last_review", "review_scores_rating", "review_scores_accuracy",
#     "review_scores_cleanliness", "review_scores_checkin", "review_scores_communication",
#     "review_scores_location", "review_scores_value", "instant_bookable", "calculated_host_listings_count",
#     "calculated_host_listings_count_entire_homes", "calculated_host_listings_count_private_rooms",
#     "calculated_host_listings_count_shared_rooms", "reviews_per_month", "host_verification_1",
#     "host_verification_2", "host_verification_3"
# ]

# # Comparação de colunas
# if colunas_listings == colunas_sql:
#     print("Os nomes das colunas estão na ordem correta!")
# else:
#     print("Os nomes das colunas ou a ordem não coincidem.")
#     # Exibir as diferenças
#     print("Colunas do DataFrame:", colunas_listings)
#     print("Colunas esperadas:", colunas_sql)


In [ ]:
# print("Número de colunas no INSERT:", insert_query.count('%s'))
# print("Número de colunas em data_to_insert:", len(data_to_insert[0]))


In [ ]:
# pd.options.display.max_rows = None
# print(listings.dtypes)

In [ ]:
# for column in listings.columns:
#     print(f"Coluna: {column}, Tipo de dados: {type(listings[column].iloc[0])}")

In [ ]:
# print(data_to_insert[0])  # Verifica os tipos da primeira linha


Fechar conexão e cursores

In [56]:
#fechar os cursores
cur_listings.close()
cur_calendar.close()
cur_review.close()

# Fechar a conexão com o database
conn.close()

Transformação de Dados - Camada Gold:
- Crie a camada "gold" de dados, aplicando agregações especializadas, como médias de preços por propriedade, por período, e outras agregações especializadas.
- Armazene os dados da camada "gold" no mesmo banco de dados PostgreSQL, mantendo a estrutura de dados otimizada para consultas analíticas.

In [57]:
from datetime import datetime

In [80]:
# Estabelecer conexão com o PostgreSQL
try:
    conn = psycopg2.connect(
        host="localhost",
        database="bronze_layer",
        user="postgres",
        password="7596"
    )
    print("Conexão bem-sucedida!")
except Exception as e:
    print("Erro ao conectar:", e)

# Criar cursor
cursor = conn.cursor()


Conexão bem-sucedida!


In [81]:

# 1. Métricas de Propriedades
def create_property_metrics():
    query = """
    CREATE TABLE IF NOT EXISTS gold_property_metrics AS
    WITH price_stats AS (
        SELECT
            l.id,
            l.property_type,
            l.room_type,
            l.neighbourhood_cleansed,
            l.accommodates,
            ROUND(AVG(l.price)::numeric, 2) as avg_price,
            ROUND(MIN(l.price)::numeric, 2) as min_price,
            ROUND(MAX(l.price)::numeric, 2) as max_price,
            ROUND(AVG(l.review_scores_rating)::numeric, 2) as avg_rating,
            COUNT(r.id) as total_reviews,
            ROUND(AVG(l.reviews_per_month)::numeric, 2) as avg_reviews_per_month
        FROM listings_silver l
        LEFT JOIN reviews_silver r ON l.id = r.listing_id
        GROUP BY
            l.id,
            l.property_type,
            l.room_type,
            l.neighbourhood_cleansed,
            l.accommodates
    )
    SELECT
        *,
        NTILE(4) OVER (ORDER BY avg_price) as price_quartile,
        CASE
            WHEN avg_rating >= 4.5 THEN 'Excelente'
            WHEN avg_rating >= 4.0 THEN 'Muito Bom'
            WHEN avg_rating >= 3.5 THEN 'Bom'
            ELSE 'Regular'
        END as rating_category
    FROM price_stats;
    """
    cursor.execute(query)
    conn.commit()


In [82]:

# 2. Métricas de Ocupação
def create_occupancy_metrics():
    query = """
    CREATE TABLE IF NOT EXISTS gold_occupancy_metrics AS
    WITH occupancy_stats AS (
        SELECT
            c.listing_id,
            DATE_TRUNC('month', c.date) as month,
            COUNT(*) as total_days,
            SUM(CASE WHEN c.available = false THEN 1 ELSE 0 END) as booked_days,
            ROUND(AVG(c.price)::numeric, 2) as avg_daily_price
        FROM calendar_silver c
        GROUP BY c.listing_id, DATE_TRUNC('month', c.date)
    )
    SELECT
        os.*,
        ROUND((booked_days::float / total_days * 100)::numeric, 2) as occupancy_rate,
        ROUND((booked_days::float * avg_daily_price)::numeric, 2) as estimated_revenue
    FROM occupancy_stats os;
    """
    cursor.execute(query)
    conn.commit()


In [83]:

# 3. Métricas do Host
def create_host_metrics():
    query = """
    CREATE TABLE IF NOT EXISTS gold_host_metrics AS
    SELECT
        h.host_id,
        h.host_name,
        h.host_location,
        COUNT(DISTINCT h.id) as total_properties,
        ROUND(AVG(h.price)::numeric, 2) as avg_property_price,
        ROUND(AVG(h.review_scores_rating)::numeric, 2) as avg_rating,
        SUM(h.number_of_reviews) as total_reviews,
        ROUND(AVG(CASE WHEN h.host_is_superhost THEN 1 ELSE 0 END)::numeric, 2) as superhost_ratio,
        ROUND(AVG(h.host_response_rate)::numeric, 2) as avg_response_rate,
        ROUND(AVG(h.host_acceptance_rate)::numeric, 2) as avg_acceptance_rate
    FROM listings_silver h
    GROUP BY
        h.host_id,
        h.host_name,
        h.host_location;
    """
    cursor.execute(query)
    conn.commit()


In [84]:

# 4. Métricas de Reviews e Satisfação
def create_review_metrics():
    query = """
    CREATE TABLE IF NOT EXISTS gold_review_metrics AS
    WITH review_stats AS (
        SELECT
            l.id as listing_id,
            l.neighbourhood_cleansed,
            COUNT(r.id) as total_reviews,
            ROUND(AVG(l.review_scores_rating)::numeric, 2) as avg_rating,
            ROUND(AVG(l.review_scores_cleanliness)::numeric, 2) as avg_cleanliness,
            ROUND(AVG(l.review_scores_location)::numeric, 2) as avg_location,
            ROUND(AVG(l.review_scores_value)::numeric, 2) as avg_value,
            ROUND(AVG(l.review_scores_communication)::numeric, 2) as avg_communication,
            MAX(r.date) as last_review_date,
            MIN(r.date) as first_review_date
        FROM listings_silver l
        LEFT JOIN reviews_silver r ON l.id = r.listing_id
        GROUP BY l.id, l.neighbourhood_cleansed
    )
    SELECT
        *,
        (DATE_PART('day', last_review_date - first_review_date))::integer as days_active,
        CASE
            WHEN avg_rating >= 4.5 THEN 'Alta Satisfação'
            WHEN avg_rating >= 4.0 THEN 'Boa Satisfação'
            WHEN avg_rating >= 3.5 THEN 'Satisfação Regular'
            ELSE 'Necessita Atenção'
        END as satisfaction_level
    FROM review_stats;
    """
    cursor.execute(query)
    conn.commit()


In [ ]:

# 5. Métricas de Sazonalidade
def create_seasonality_metrics():
    query = """
    CREATE TABLE IF NOT EXISTS gold_seasonality_metrics AS
    SELECT
        listing_id,
        DATE_PART('month', date) as month,
        DATE_PART('year', date) as year,
        ROUND(AVG(price)::numeric, 2) as avg_price,
        COUNT(*) as total_days,
        SUM(CASE WHEN available = false THEN 1 ELSE 0 END) as booked_days,
        ROUND((SUM(CASE WHEN available = false THEN 1 ELSE 0 END)::float / COUNT(*)::float * 100)::numeric, 2) as occupancy_rate
    FROM calendar_silver
    GROUP BY listing_id, DATE_PART('month', date), DATE_PART('year', date)
    ORDER BY listing_id, year, month;
    """
    cursor.execute(query)
    conn.commit()

# Executar todas as transformações
def create_gold_layer():
    print("Criando camada Gold...")

    print("1. Criando métricas de propriedades...")
    create_property_metrics()

    print("2. Criando métricas de ocupação...")
    create_occupancy_metrics()

    print("3. Criando métricas de hosts...")
    create_host_metrics()

    print("4. Criando métricas de reviews...")
    create_review_metrics()

    print("5. Criando métricas de sazonalidade...")
    create_seasonality_metrics()

    print("Camada Gold criada com sucesso!")

    # Fechar conexões
    cursor.close()
    conn.close()

# Executar a criação da camada Gold
if __name__ == "__main__":
    create_gold_layer()

In [79]:
cursor.close()
conn.close()

In [86]:
create_gold_layer()

Criando camada Gold...
1. Criando métricas de propriedades...
2. Criando métricas de ocupação...
3. Criando métricas de hosts...
4. Criando métricas de reviews...


UndefinedFunction: function date_part(unknown, integer) does not exist
LINE 21:         (DATE_PART('day', last_review_date - first_review_da...
                  ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.
